In [2]:
import pandas as pd
import re
import yaml
import sqlparse
import os
import pandas as pd
import requests
from IPython.display import display, Markdown


In [3]:
def add_repo_root_path():
    import os
    import sys
    repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    if repo_root not in sys.path:
        sys.path.append(repo_root)
        
add_repo_root_path()
from src import generate_knowledge

In [4]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', 10) 

## Read dbs

In [5]:
local_dbt_repo = ''
online_dbt_repo = 'https://github.com/dbt-labs/jaffle-shop'

# Use local repo?
if False:
    repo_path = local_dbt_repo
else:
    repo_path = online_dbt_repo

is_online = generate_knowledge.is_online_repo(repo_path)
_, repo_name = generate_knowledge.extract_owner_and_repo(repo_path)
print(repo_name)

jaffle-shop


In [6]:
dbt_models_df = pd.read_csv('../data/dbt_models_' + repo_name + '.csv')
dbt_project_df = pd.read_csv('../data/dbt_project_' + repo_name + '.csv')
display(dbt_models_df.sample(3))
display(dbt_project_df.sample(3))

,path,name,extension,sql_code,config,materialized,is_snapshot,has_jinja_code,model_category,vertical,yml_code,tests,has_tests,sql_ids,has_select_all_in_last_select,has_group_by,primary_key,filters,is_filtered,macros,has_macros,parent_models,is_source_model,source,children_models,is_end_model,model_description,jinja_description
1,models/staging/stg_locations.sql,stg_locations.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,NaN,NaN,False,False,stg,locations,"{'models': [{'name': 'stg_locations', 'descrip...","{'columns': {'location_id': ['not_null', 'uniq...",True,['location_id'],True,False,location_id,NaN,False,['dbt.date_trunc'],True,[],True,['ecom.raw_stores'],['locations'],False,"""Extracts a list of open store locations from ...",NaN
9,models/marts/order_items.sql,order_items.sql,.sql,WITH order_items AS\n (SELECT *\n FROM {{ r...,NaN,NaN,False,False,other,order_items,"{'models': [{'name': 'order_items', 'columns':...","{'columns': {'order_item_id': ['not_null', 'un...",True,"['product_id', 'order_id']",True,True,order_item_id,"['order_items.order_id = orders.order_id', 'or...",True,NaN,False,"['stg_order_items', 'stg_orders', 'stg_product...",False,NaN,['orders'],False,"""Combines order item details from 'stg_order_i...",NaN
10,models/marts/orders.sql,orders.sql,.sql,WITH orders AS\n (SELECT *\n FROM {{ ref('s...,NaN,NaN,False,False,other,orders,"{'models': [{'name': 'orders', 'description': ...","{'columns': {'order_id': ['not_null', 'unique'...",True,"['order_item_id', 'customer_id', 'order_id']",True,True,order_id,['orders.order_id = order_items_summary.order_...,True,NaN,False,"['stg_orders', 'order_items']",False,NaN,['customers'],False,"""Aggregates order data from the 'stg_orders' a...",NaN


,path,name,extension,code,is_seed,is_macro,is_test,packages,description
11,jaffle-data/raw_supplies.csv,raw_supplies.csv,.csv,id name cost per...,False,False,False,NaN,NaN
5,macros/generate_schema_name.sql,generate_schema_name.sql,.sql,{% macro generate_schema_name(custom_schema_na...,False,True,False,NaN,Generates a schema name based on the resource ...
0,Taskfile.yml,Taskfile.yml,.yml,version: '3'\nsilent: true\nvars:\n YEARS: 6\...,False,False,False,NaN,NaN


In [7]:
def merge_dbt_models_and_project_dfs(dbt_models_df, dbt_project_df):
    dbt_models_df['knowledge_type'] = 'models'
    dbt_project_df['knowledge_type'] = 'project'

    dbt_models_df.rename(columns = {'sql_code':'code'}, inplace = True)

    all_columns = set(dbt_models_df.columns).union(set(dbt_project_df.columns))

    for col in all_columns:
        if col not in dbt_models_df:
            dbt_models_df[col] = None
        if col not in dbt_project_df:
            dbt_project_df[col] = None

    merged_df = pd.concat([dbt_models_df, dbt_project_df], ignore_index=True)
    columns_order = ['knowledge_type'] + [col for col in merged_df.columns if col != 'knowledge_type']
    return merged_df[columns_order]

In [8]:
dbt_repo_knowledge_df = merge_dbt_models_and_project_dfs(dbt_models_df, dbt_project_df)

## LangChain

#### Embeddings and Documents db

In [9]:
def combine_contextual_fields(row):
    combined = f"""        
        Code:
        {row['code'] if pd.notna(row['code']) else 'N/A'}

        YML Code:
        {row['yml_code'] if pd.notna(row['yml_code']) else 'N/A'}

        Description for project files:
        {row['description'] if pd.notna(row['description']) else 'N/A'}

        dbt Model description:
        {row['model_description'] if pd.notna(row['model_description']) else 'N/A'}

        Jinja inside the dbt model description:
        {row['jinja_description'] if pd.notna(row['jinja_description']) else 'N/A'}
    """
    return combined.strip()

dbt_repo_knowledge_df['contextual_info'] = dbt_repo_knowledge_df.apply(combine_contextual_fields, axis=1)

In [10]:
dbt_repo_knowledge_df

,knowledge_type,path,name,extension,code,config,materialized,is_snapshot,has_jinja_code,model_category,vertical,yml_code,tests,has_tests,sql_ids,has_select_all_in_last_select,has_group_by,primary_key,filters,is_filtered,macros,has_macros,parent_models,is_source_model,source,children_models,is_end_model,model_description,jinja_description,is_test,description,packages,is_macro,is_seed,contextual_info
0,models,models/staging/stg_customers.sql,stg_customers.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,NaN,NaN,False,False,stg,customers,"{'models': [{'name': 'stg_customers', 'descrip...","{'columns': {'customer_id': ['not_null', 'uniq...",True,['customer_id'],True,False,customer_id,NaN,False,NaN,False,[],True,['ecom.raw_customers'],['customers'],False,"""Extracts customer data from the 'raw_customer...",NaN,None,None,None,None,None,Code:\n WITH SOURCE AS\n (SELECT *\n ...
1,models,models/staging/stg_locations.sql,stg_locations.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,NaN,NaN,False,False,stg,locations,"{'models': [{'name': 'stg_locations', 'descrip...","{'columns': {'location_id': ['not_null', 'uniq...",True,['location_id'],True,False,location_id,NaN,False,['dbt.date_trunc'],True,[],True,['ecom.raw_stores'],['locations'],False,"""Extracts a list of open store locations from ...",NaN,None,None,None,None,None,Code:\n WITH SOURCE AS\n (SELECT *\n ...
2,models,models/staging/stg_order_items.sql,stg_order_items.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,NaN,NaN,False,False,stg,order_items,"{'models': [{'name': 'stg_order_items', 'descr...","{'columns': {'order_item_id': ['not_null', 'un...",True,"['product_id', 'order_item_id', 'order_id']",True,False,order_item_id,NaN,False,NaN,False,[],True,['ecom.raw_items'],['order_items'],False,"""Extracts individual food and drink items from...",NaN,None,None,None,None,None,Code:\n WITH SOURCE AS\n (SELECT *\n ...
3,models,models/staging/stg_orders.sql,stg_orders.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,NaN,NaN,False,False,stg,orders,"{'models': [{'name': 'stg_orders', 'descriptio...","{'columns': {'order_id': ['not_null', 'unique'...",True,"['location_id', 'store_id', 'customer_id', 'or...",True,False,order_id,NaN,False,"['cents_to_dollars', 'dbt.date_trunc']",True,[],True,['ecom.raw_orders'],"['order_items', 'orders']",False,Retrieves order data from the 'ecom.raw_orders...,NaN,None,None,None,None,None,Code:\n WITH SOURCE AS\n (SELECT *\n ...
4,models,models/staging/stg_products.sql,stg_products.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,NaN,NaN,False,False,stg,products,"{'models': [{'name': 'stg_products', 'descript...","{'columns': {'product_id': ['not_null', 'uniqu...",True,['product_id'],True,False,product_id,NaN,False,['cents_to_dollars'],True,[],True,['ecom.raw_products'],"['order_items', 'products']",False,Retrieves product data from the 'raw_products'...,NaN,None,None,None,None,None,Code:\n WITH SOURCE AS\n (SELECT *\n ...
5,models,models/staging/stg_supplies.sql,stg_supplies.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,NaN,NaN,False,False,stg,supplies,"{'models': [{'name': 'stg_supplies', 'descript...","{'columns': {'supply_uuid': ['not_null', 'uniq...",True,"['product_id', 'supply_id']",True,False,supply_uuid,NaN,False,"['cents_to_dollars', 'dbt_utils.generate_surro...",True,[],True,['ecom.raw_supplies'],"['order_items', 'supplies']",False,Retrieves supply expense data from the 'raw_su...,NaN,None,None,None,None,None,Code:\n WITH SOURCE AS\n (SELECT *\n ...
6,models,models/marts/customers.sql,customers.sql,.sql,WITH customers AS\n (SELECT *\n FROM {{ ref...,NaN,NaN,False,False,other,customers,"{'models': [{'name': 'customers', 'description...","{'columns': {'customer_id': ['not_null', 'uniq...",True,"['customer_id', 'order_id']",True,True,customer_id,['customers.customer_id = customer_orders_summ...,True,NaN,False,"['stg_customers', 'orders']",False,NaN,[],True,"""Aggregates customer data from the 'stg_custom..

In [11]:
generate_knowledge.add_repo_root_path()
import openai_setup

OPENAI_API_KEY = openai_setup.conf['key']
OPENAI_PROJECT = openai_setup.conf['project']
OPENAI_ORGANIZATION = openai_setup.conf['organization']
DEFAULT_LLM_MODEL = "gpt-4o-mini"

In [12]:
from langchain.schema import Document

documents = dbt_repo_knowledge_df.apply(
    lambda row: Document(
        page_content=row["contextual_info"],
        metadata={
            "knowledge_type": row["knowledge_type"],
            "name": row["name"],
            "path": row["source"],
            "source": row["path"],
            "parents": row["parent_models"],
            "children": row["children_models"]
        }
    ), axis=1
).tolist()

In [13]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore = FAISS.from_documents(documents, embeddings)

In [14]:
vectorstore

#### Test simple Retriever

In [15]:
from langchain.chains import RetrievalQA, LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.schema.runnable import RunnableSequence

llm = ChatOpenAI(model=DEFAULT_LLM_MODEL, temperature=0.1, openai_api_key=OPENAI_API_KEY, openai_organization = OPENAI_ORGANIZATION)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [16]:
template = """
    You are a dbt expert. Based on the following context, answer the question concisely and accurately:
    {context}

    Question: {question}
"""
qa_prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": qa_prompt},
)

In [17]:
query = "Que modelos se veran fectados si cambio el nombre de customer_id en stg_customers"
result = qa_chain.invoke({"query": query})

print(result["result"])
print(result["source_documents"])

Si cambias el nombre de `customer_id` en el modelo `stg_customers`, los siguientes modelos se verán afectados:

1. **Modelo `customers`**: Este modelo hace referencia a `stg_customers` y utiliza `customer_id` en su lógica de unión y en la selección de columnas.

2. **Modelo `customer_orders_summary`**: Este modelo también utiliza `customer_id` para agrupar y resumir datos de pedidos.

Cualquier modelo que dependa de `stg_customers` y utilice `customer_id` necesitará ser actualizado para reflejar el nuevo nombre.
[Document(id='78761315-7033-47de-b529-e646063fff52', metadata={'knowledge_type': 'models', 'name': 'stg_customers.sql', 'path': "['ecom.raw_customers']", 'source': 'models/staging/stg_customers.sql', 'parents': '[]', 'children': "['customers']"}, page_content='Code:\n        WITH SOURCE AS\n  (SELECT *\n   FROM {{ source(\'ecom\', \'raw_customers\') }}), renamed AS\n  (SELECT ----------  ids\n id AS customer_id, ---------- text\n name AS customer_name\n   FROM SOURCE)\nSELECT *

#### Agent test

In [18]:
# Retrieval Tool
retrieval_tool = Tool(
    name="Retrieve Info",
    func=lambda query: retriever.get_relevant_documents(query),
    description="Retrieve relevant context from the knowledge base.",
)

# Task Execution Prompt
task_prompt = PromptTemplate(
    template="""
        You are a dbt expert. Based on the context provided, perform the task below step-by-step:
        
        Task: {question}
        Context: {context}
    """,
    input_variables=["context", "question"],
)

# QA Chain for Task Execution
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": task_prompt},
)

# Task Execution Tool
task_tool = Tool(
    name="DBT Task Executor",
    func=lambda query: qa_chain.invoke({"query": query}),
    description="Execute dbt-related tasks like adding columns, creating models, or verifying dependencies.",
)

# Agent Configuration
tools = [retrieval_tool, task_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
)

/var/folders/hv/ckh3m6gn1sd45q2qctrqcwzh0000gn/T/ipykernel_80642/586826549.py:38: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(


In [19]:
# Example Query
query = "Add a new column 'customer_age' to the 'stg_customers' model. The new column is available in the customer sources"

# Run the Agent
result = agent.run(query)
print("Agent Output:", result)

/var/folders/hv/ckh3m6gn1sd45q2qctrqcwzh0000gn/T/ipykernel_80642/795793740.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run(query)




> Entering new AgentExecutor chain...
I need to add a new column 'customer_age' to the 'stg_customers' model. This requires executing a dbt task to modify the model and include the new column from the customer sources. 

Action: DBT Task Executor  
Action Input: Add a new column 'customer_age' to the 'stg_customers' model using the customer sources.  

Observation: {'query': "Add a new column 'customer_age' to the 'stg_customers' model using the customer sources.  \n", 'result': 'To add a new column `customer_age` to the `stg_customers` model using the customer sources, we will follow these steps:\n\n### Step 1: Modify the SQL Code for `stg_customers`\n\nWe need to update the SQL code in the `stg_customers` model to include the new column `customer_age`. Assuming that the `raw_customers` source contains a date of birth or a similar field from which we can calculate the age, we will add this logic to the SQL.\n\nHere’s how the updated SQL code might look:\n\n```sql\nWITH SOURCE AS (\n

#### Start agent chain

In [24]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

class Step:
    """Base class for a step in the processing pipeline."""
    def __init__(self, name):
        self.name = name

    def execute(self, input_data, context):
        raise NotImplementedError("Each step must implement the execute method.")


class ReflectionStep(Step):
    """Reflect on the user's request."""
    def __init__(self, llm):
        super().__init__("Reflection")
        self.llm = llm
        self.prompt_template = PromptTemplate(
            input_variables=["request"],
            template="Reflexiona sobre esta petición: {request}. Responde con lo que crees que se debe hacer."
        )

    def execute(self, input_data, context):
        print(f"Step: {self.name} | Input: {input_data}")
        prompt = self.prompt_template.format(request=input_data)
        # Pasar directamente como string
        reflection = self.llm.predict(prompt)
        context["reflection"] = reflection
        print(f"Step: {self.name} | Output: {reflection}")
        return reflection


class PlanStep(Step):
    """Plan the steps needed based on reflection."""
    def __init__(self, llm, retriever):
        super().__init__("Plan")
        self.llm = llm
        self.retriever = retriever
        self.prompt_template = PromptTemplate(
            input_variables=["reflection"],
            template="Con base en esta reflexión: {reflection}, crea un plan de pasos para resolver la petición."
        )

    def execute(self, input_data, context):
        print(f"Step: {self.name} | Input: {input_data}")
        reflection = context.get("reflection", "")
        prompt = self.prompt_template.format(reflection=reflection)

        # Recuperar documentos con el método adecuado
        documents = self.retriever.get_relevant_documents(input_data)
        context["documents"] = documents
        print(f"Step: {self.name} | Retrieved Documents: {documents}")

        # Pasar el prompt directamente como string
        plan = self.llm.predict(prompt)
        context["plan"] = plan
        print(f"Step: {self.name} | Output: {plan}")
        return plan


class ChatPipeline:
    """Manages the pipeline of steps."""
    def __init__(self, steps):
        self.steps = steps

    def execute(self, input_data):
        context = {}
        for step in self.steps:
            print(f"\nExecuting step: {step.name}")
            result = step.execute(input_data, context)
            print(f"Finished step: {step.name} | Result: {result}\n")
        return context


def main(user_input, llm, retriever):
    # Define the steps
    steps = [
        ReflectionStep(llm),
        PlanStep(llm, retriever),
    ]

    # Create the pipeline
    pipeline = ChatPipeline(steps)

    context = pipeline.execute(user_input)
    print("Context at the end of pipeline:", context)

    return context


In [25]:
user_input = "Add a new column 'customer_age' to the 'stg_customers' model. The new column is available in the customer sources"
llm_results = main(user_input, llm, retriever)

Chat bot initialized. Type your request or 'exit' to quit.

Executing step: Reflection
Step: Reflection | Input: Add a new column 'customer_age' to the 'stg_customers' model. The new column is available in the customer sources


/var/folders/hv/ckh3m6gn1sd45q2qctrqcwzh0000gn/T/ipykernel_80642/1101144321.py:27: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  reflection = self.llm.predict(prompt)


Step: Reflection | Output: Para abordar la petición de agregar una nueva columna 'customer_age' al modelo 'stg_customers', es importante seguir un enfoque estructurado. Aquí hay algunos pasos que se deben considerar:

1. **Revisión de la Fuente de Datos**: Verificar que la información sobre la edad del cliente esté disponible en las fuentes de datos. Esto puede implicar revisar los archivos, bases de datos o APIs de donde se extraen los datos de los clientes.

2. **Definición del Tipo de Dato**: Determinar el tipo de dato que se utilizará para la nueva columna 'customer_age'. Generalmente, esto será un número entero (integer) que represente la edad del cliente.

3. **Modificación del Modelo**: Actualizar el modelo 'stg_customers' para incluir la nueva columna. Esto puede implicar modificar el esquema de la base de datos o el archivo de definición del modelo, dependiendo de la tecnología que se esté utilizando (por ejemplo, SQL, dbt, etc.).

4. **Transformación de Datos**: Si la edad no

In [26]:
llm_results

{'reflection': "Para abordar la petición de agregar una nueva columna 'customer_age' al modelo 'stg_customers', es importante seguir un enfoque estructurado. Aquí hay algunos pasos que se deben considerar:\n\n1. **Revisión de la Fuente de Datos**: Verificar que la información sobre la edad del cliente esté disponible en las fuentes de datos. Esto puede implicar revisar los archivos, bases de datos o APIs de donde se extraen los datos de los clientes.\n\n2. **Definición del Tipo de Dato**: Determinar el tipo de dato que se utilizará para la nueva columna 'customer_age'. Generalmente, esto será un número entero (integer) que represente la edad del cliente.\n\n3. **Modificación del Modelo**: Actualizar el modelo 'stg_customers' para incluir la nueva columna. Esto puede implicar modificar el esquema de la base de datos o el archivo de definición del modelo, dependiendo de la tecnología que se esté utilizando (por ejemplo, SQL, dbt, etc.).\n\n4. **Transformación de Datos**: Si la edad no es